Chinese Word2vec
=============

The goal of this assignment is to train a Word2Vec using gensim over zhwiki(http://dumps.wikimedia.org/zhwiki/latest/) data and show the result by TSNE.
------------

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import logging
import os
import re
import codecs
import jieba
from opencc import OpenCC
from gensim import models
from gensim.models import word2vec 
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from matplotlib import pylab
from pylab import mpl

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Download the data from the source website if necessary.

In [2]:
url = 'http://dumps.wikimedia.org/zhwiki/latest/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('zhwiki-latest-pages-articles1.xml-p1p231819.bz2', 200471167)

Found and verified zhwiki-latest-pages-articles1.xml-p1p231819.bz2


# Extract data by WikiExtractor.py

# Data Processing

In [3]:
def cut(string): return list(jieba.cut(string))

In [4]:
def get_stopwords(filename = "D:/Code/Pycharm/Data_source/chinese_stopwords.txt"):
    stopwords_dic = open(filename, encoding= 'utf-8')
    stopwords = stopwords_dic.readlines()
    stopwords = [w.strip() for w in stopwords]
    stopwords_dic.close()
    print(stopwords)
    stopwords.append('\r\n')
    return stopwords

In [5]:
stopwords = get_stopwords()

['，', '的', '。', '、', '在', '了', '是', '', '“', '”', '和', '年', '月', '：', '也', '）', '为', '（', '有', '%', '日', '将', '中', '-', '到', '与', '对', ':', '', '上', '都', '等', '不', '他', '》', '《', '就', '但', '我', '而', '这', '会', '并', '；', '被', '后', '人', '从', '还', '1', '3', '6', '以', '新', '说', '7', '2', '要', '5', '？', '更', '于', '个', '10', '大', '时', '4', '多', '/', '让', '其', ')', '(', '很', '及', '下', '', '能', '—', '或', '该', '她', '比', '8', '元', '12', '已', '向', '做', '来', '前', '由', '好', '.', '称', '给', '最', '11', '·', '据', '着', '又', '至', '9', '20', '！', '[', ']', '去', '可', '把', '则', '', '一', '地', '高', '吗', '30', '所', '分', '较', '内', '第', '里', '占', '过', '15', '曾', '"', '再', '人民日报', '新闻网', '它', '况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或者', '极了', 

In [6]:
def read(filename): 
    sline = ''
    with codecs.open(filename,'r','utf-8') as f:
        for line in f:
            sline += line.strip()
    return sline

In [7]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

In [8]:
def cleanstopword(words):
    words = [w for w in words if len(w)>1 and not re.match('^[a-z|A-Z|0-9|.]*$',w)]
    words = [w for w in words if w not in stopwords]
    return words 

In [9]:
def convert2simple(word):
    openCC = OpenCC('tw2sp')
    return openCC.convert(word)

In [10]:
def wiki2txt(filename):
    output = open('wiki_seg.txt', 'a', encoding='utf-8')
    for root, dirs, files in os.walk(filename):
        for filename in files:
            file_path = root + '/' + filename
            content = read(file_path)
            clean = cleanhtml(content)
            converted = convert2simple(clean)
            cut_line = cleanstopword(cut(converted))
            line = ' '.join(cut_line)
            output.write(line)
    output.close()

In [ ]:
wiki2txt("D:/Code/zhwiki-gensim-word2vec/AA")

# train a Word2Vec

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.LineSentence("wiki_seg.txt")
model = word2vec.Word2Vec(sentences, size=250)
model.save("word2vec.model")

# Plot result

In [15]:
mpl.rcParams['font.sans-serif'] = ['FangSong'] # Chinese show
mpl.rcParams['axes.unicode_minus'] = False 

In [11]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15,15),dpi=600)  
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',ha='right', va='bottom')
    pylab.show()

In [12]:
def show_word(word_start,word_end):
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    two_d_embeddings = tsne.fit_transform(X[word_start:word_end, :])
    plot(two_d_embeddings, wordVocab[word_start:word_end])

In [13]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = models.Word2Vec.load('word2vec.model')

wordVocab = [k for (k,v) in model.wv.vocab.items()]
X = model[model.wv.vocab]

2018-05-31 20:54:31,108 : INFO : loading Word2Vec object from word2vec.model
2018-05-31 20:54:31,396 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2018-05-31 20:54:31,397 : INFO : loading vectors from word2vec.model.wv.vectors.npy with mmap=None
2018-05-31 20:54:31,536 : INFO : setting ignored attribute vectors_norm to None
2018-05-31 20:54:31,537 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2018-05-31 20:54:31,537 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2018-05-31 20:54:31,538 : INFO : loading syn1neg from word2vec.model.trainables.syn1neg.npy with mmap=None
2018-05-31 20:54:31,596 : INFO : setting ignored attribute cum_table to None
2018-05-31 20:54:31,597 : INFO : loaded word2vec.model
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [16]:
show_word(1000,1100)

# Other test

In [ ]:
word = '爱情'
res = model.wv.most_similar(word,topn = 10)
for item in res:
	print(item[0]+","+str(item[1]))
 
model.wv.most_similar(positive=['女人', '妈妈'], negative=['男人'], topn=1)

model.wv.doesnt_match("苹果 西瓜 香蕉 牛肉".split())

model.wv.similarity('男人', '女人')  